In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
## bUILT-iN pACKAGES
import pickle
from collections import defaultdict
import json
import numpy as np
import math
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True)
%matplotlib inline
from IPython.display import display

## lAST sUMMER
from romtoslp import *
from sentences import *
from DCS import *
import MatDB

## lAST yEAR
# from word_definite import *
# from nnet import *
# from heap_n_PrimMST import *
# from word_definite import *

In [3]:
import winsound
def playBeep():
    for i in range(3):
        winsound.Beep(2200, 300)
        winsound.Beep(2600, 300)

In [4]:
loaded_SKT = pickle.load(open('../Simultaneous_CompatSKT_10K.p', 'rb'), encoding=u'utf-8')
loaded_DCS = pickle.load(open('../Simultaneous_DCS_10K.p', 'rb'), encoding=u'utf-8')

In [5]:
dataset_4k_1k = pickle.load(open('../SmallDataset_4K_1K.p', 'rb'))
TrainFiles = dataset_4k_1k['TrainFiles']
TestFiles = dataset_4k_1k['TestFiles']

In [6]:
matDB = MatDB.MatDB()

In [7]:
# from MatDB import *
import word_definite as WD
from heap_n_PrimMST import *
from nnet import *
"""
################################################################################################
######################  CREATE SEVERAL DATA STRUCTURES FROM SENTENCE/DCS  ######################
###########################  NODELIST, ADJACENCY LIST, GRAPH, HEAP #############################
################################################################################################
"""
def GetTrainingKit(sentenceObj, dcsObj):
    nodelist = GetNodes(sentenceObj)
    
    # Nodelist with only the correct_nodes
    nodelist2 = GetNodes(sentenceObj)
    nodelist2_to_correct_mapping = {}
    nodelist_correct = []
    search_key = 0
    first_key = 0
    for chunk_id in range(len(dcsObj.lemmas)):
        while nodelist2[first_key].chunk_id != chunk_id:
            first_key += 1
        for j in range(len(dcsObj.lemmas[chunk_id])):
            search_key = first_key
            while (nodelist2[search_key].lemma != rom_slp(dcsObj.lemmas[chunk_id][j])) or (nodelist2[search_key].cng != dcsObj.cng[chunk_id][j]):
                search_key += 1
                if search_key >= len(nodelist2) or nodelist2[search_key].chunk_id > chunk_id:
                    break
    #         print((rom_slp(dcsObj.lemmas[chunk_id][j]), dcsObj.cng[chunk_id][j]))
    #         print(nodelist[search_key])
            nodelist2_to_correct_mapping[len(nodelist_correct)] = search_key
            nodelist_correct.append(nodelist2[search_key])
    return (nodelist, nodelist_correct, nodelist2_to_correct_mapping)
    

def GetGraph(nodelist, neuralnet):
    if not neuralnet.outer_relu:
        conflicts_Dict = Get_Conflicts(nodelist)

        featVMat = Get_Feat_Vec_Matrix(nodelist, conflicts_Dict)

        (WScalarMat, SigmoidGateOutput) = Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat, nodelist, conflicts_Dict, neuralnet)
        return (conflicts_Dict, featVMat, WScalarMat, SigmoidGateOutput)
    else:
        conflicts_Dict = Get_Conflicts(nodelist)

        featVMat = Get_Feat_Vec_Matrix(nodelist, conflicts_Dict)

        WScalarMat = Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat, nodelist, conflicts_Dict, neuralnet)
        return (conflicts_Dict, featVMat, WScalarMat)

In [100]:
import time
st = str(int((time.time() * 1e6) % 1e13))
log_name = 'logs/train_nnet_t{}.out'.format(st)
p_name = 'outputs/train_nnet_t{}.p'.format(st)
print(p_name)
# outputs/train_nnet_t9543323920940.p - 80%

outputs/train_nnet_t9745407094833.p


In [17]:
trainingStatus = defaultdict(lambda: bool(False))

In [47]:
"""
################################################################################################
##############################  MAIN FUNCTION  #################################################
################################################################################################
"""

def train(loaded_SKT, loaded_DCS, n_trainset = -1):
    # Train
    filePerBatch = 20
    iterationPerBatch = 10
    if n_trainset == -1:
        totalBatchToTrain = 20
    else:
        totalBatchToTrain = math.ceil(n_trainset/filePerBatch)
    
    for iterout in range(totalBatchToTrain):
        # Change current batch
        trainer.Save(p_name)
        print('Batch: ', iterout)
        files_for_batch = TrainFiles[iterout*filePerBatch:(iterout + 1)*filePerBatch]
        print(files_for_batch)
        # trainer.Load('outputs/neuralnet_trained.p')
        
        # Run few times on same set of files
        for iterin in range(iterationPerBatch):
            print('ITERATION IN', iterin)        
            for fn in files_for_batch:
                sentenceObj = loaded_SKT[fn]
                dcsObj = loaded_DCS[fn]
                if trainingStatus[sentenceObj.sent_id]:
                    continue
                # trainer.Save('outputs/saved_trainer.p')
                try:
                    trainer.Train(sentenceObj, dcsObj)
                except (IndexError, KeyError) as e:
                    print('\x1b[31mFailed: {} \x1b[0m'.format(sentenceObj.sent_id))
    trainer.Save(p_name)
                
def test(loaded_SKT, loaded_DCS, n_testSet = -1, _testFiles = None):
    total_lemma = 0;
    correct_lemma = 0;

    total_word = 0;
    total_output_nodes = 0
    correct_word = 0;
    file_counter = 0
    if _testFiles is None:
        if n_testSet == -1:
            _testFiles = TestFiles
        else:
            _testFiles = TestFiles[0:n_testSet]
    else:
        if n_testSet == -1:
            _testFiles = _testFiles
        else:
            _testFiles = _testFiles[0:n_testSet]
    for fn in _testFiles:
        if file_counter % 100 == 0:
            print(file_counter,' Checkpoint... ')
        file_counter += 1
        sentenceObj = loaded_SKT[fn]
        dcsObj = loaded_DCS[fn]        
        try:
            (word_match, lemma_match, n_dcsWords, n_output_nodes) = trainer.Test(sentenceObj, dcsObj)
            total_lemma += n_dcsWords
            total_word += n_dcsWords
            
            total_output_nodes += n_output_nodes            
            
            correct_lemma += lemma_match
            correct_word += word_match
        except (IndexError, KeyError) as e:
            print('Failed!')        

    print('Micro Recall[lemma]: {}'.format(100*correct_lemma/total_word))
    print('Micro Recall[word]: {}'.format(100*correct_word/total_word))
    
    print('Micro Precision[lemma]: {}'.format(100*correct_lemma/total_output_nodes))
    print('Micro Precision[word]: {}'.format(100*correct_word/total_output_nodes))
    

In [37]:
# NOW-OLD FUNCTION
# def GetLoss(_mst_adj_graph, _mask_de_correct_edges, _negLogLikelies):
#     _negLogLikelies = _negLogLikelies.copy()
#     _negLogLikelies[~_mst_adj_graph] = 0
#     _negLogLikelies[~_mask_de_correct_edges] *= -1 # BAKA!!! Check before you try to fix this again
#     return np.sum(_negLogLikelies)    

# NEW FUNCTION
def GetLoss(_mst_adj_graph, _mask_de_correct_edges, _WScalarMat):
    _WScalarMat = _WScalarMat.copy()
    _WScalarMat[_mst_adj_graph&(~_mask_de_correct_edges)] *= -1 # BAKA!!! Check before you try to fix this again
    _WScalarMat[~_mst_adj_graph] = 0
    return np.sum(_WScalarMat)

In [78]:
class Trainer:
    def __init__(self):
        self.hidden_layer_size = 400
        self._edge_vector_dim = WD._edge_vector_dim
        self._full_cnglist = list(WD.mat_cngCount_1D)
        self.neuralnet = NN(self._edge_vector_dim, self.hidden_layer_size, outer_relu=True)
        self.history = defaultdict(lambda: list())
        
    def Reset(self):
        self.neuralnet = NN(self._edge_vector_dim, self.hidden_layer_size)
        self.history = defaultdict(lambda: list())
        
    def Save(self, filename):
        #pickle.dump({'nnet': self.neuralnet, 'history': dict(self.history)}, open(filename, 'wb'))
        return
        
    
    def Load(self, filename):
        o = pickle.load(open(filename, 'rb'))
        self.neuralnet = o['nnet']
        self.history = defaultdict(lambda: list(), o['history'])    
        
    def Test(self, sentenceObj, dcsObj):
        neuralnet = self.neuralnet
        minScore = np.inf
        minMst = None
        try:
            (nodelist, nodelist_correct, _) = GetTrainingKit(sentenceObj, dcsObj)
            # nodelist = GetNodes(sentenceObj)
        except IndexError:
            print('\x1b[31mError with {} \x1b[0m'.format(sentenceObj.sent_id))
            return (0, 0, 0)
            
        conflicts_Dict = Get_Conflicts(nodelist)
        conflicts_Dict_correct = Get_Conflicts(nodelist_correct)
        
        featVMat = Get_Feat_Vec_Matrix(nodelist, conflicts_Dict)
        featVMat_correct = Get_Feat_Vec_Matrix(nodelist_correct, conflicts_Dict_correct)
        
        if not self.neuralnet.outer_relu:
            (WScalarMat, SigmoidGateOutput) = Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat, nodelist, conflicts_Dict, neuralnet)
        else:
            WScalarMat = Get_W_Scalar_Matrix_from_FeatVect_Matrix(featVMat, nodelist, conflicts_Dict, neuralnet)
        # print(WScalarMat)
        # Get all MST
        for source in range(len(nodelist)):
            (mst_nodes, mst_adj_graph) = MST(nodelist, WScalarMat, conflicts_Dict, source)
            # print('.', end = '')
            score = GetMSTWeight(mst_nodes, WScalarMat)
            if(score < minScore):
                minScore = score
                minMst = mst_nodes
        dcsLemmas = [[rom_slp(l) for l in arr]for arr in dcsObj.lemmas]
        word_match = 0
        lemma_match = 0
        n_output_nodes = 0
        for chunk_id, wdSplit in minMst.items():
            for wd in wdSplit:
                n_output_nodes += 1
                # Match lemma
                search_result = [i for i, j in enumerate(dcsLemmas[chunk_id]) if j == wd.lemma]
                if len(search_result) > 0:
                    lemma_match += 1
                # Match CNG
                for i in search_result:
                    if(dcsObj.cng[chunk_id][i] == str(wd.cng)):
                        word_match += 1
                        # print(wd.lemma, wd.cng)
                        break
        dcsLemmas = [l for arr in dcsObj.lemmas for l in arr]
#         print('\nFull Match: {}, Partial Match: {}, OutOf {}, NodeCount: {}, '.\
#               format(word_match, lemma_match, len(dcsLemmas), len(nodelist)))
        return (word_match, lemma_match, len(dcsLemmas), n_output_nodes)
    
    def Train(self, sentenceObj, dcsObj):
        # Hyperparameter for hinge loss: m
        M_hinge = 14
        try:
            (nodelist, nodelist_correct, nodelist_to_correct_mapping) = GetTrainingKit(sentenceObj, dcsObj)
        except IndexError as e:
            # print('\x1b[31mError with {} \x1b[0m'.format(sentenceObj.sent_id))
            # print(e)
            return
        
        """ CREATE A MASK FOR ALL EDGES BETWEEN CORRECT NODE PAIRS"""
        mask_de_correct_edges = np.ndarray((len(nodelist), len(nodelist)), np.bool)*False
        for n1 in nodelist_to_correct_mapping.values():
            for n2 in nodelist_to_correct_mapping.values():
                if n1 != n2:
                    mask_de_correct_edges[n1, n2] = 1
                    mask_de_correct_edges[n2, n1] = 1
                
        
        """ FOR MST OF GRAPH WITH ONLY CORRECT SET OF NODES """
        if not self.neuralnet.outer_relu:
            (conflicts_Dict_correct, featVMat_correct, WScalarMat_correct, _) = GetGraph(nodelist_correct, self.neuralnet)
        else:
            (conflicts_Dict_correct, featVMat_correct, WScalarMat_correct) = GetGraph(nodelist_correct, self.neuralnet)
        source = 0
        (mst_nodes_correct, mst_adj_graph_correct_0) = MST(nodelist_correct, WScalarMat_correct, conflicts_Dict_correct, source)
        
        
        
        """ FOR ALL POSSIBLE MST FROM THE COMPLETE GRAPH """
        if not self.neuralnet.outer_relu:
            (conflicts_Dict, featVMat, WScalarMat, SigmoidGateOutput) = GetGraph(nodelist, self.neuralnet)
        else:
            (conflicts_Dict, featVMat, WScalarMat) = GetGraph(nodelist, self.neuralnet)
        Total_Loss = 0

        Loss_2 = np.sum(WScalarMat*mask_de_correct_edges - WScalarMat*(~mask_de_correct_edges))
        
        # Convert correct spanning tree graph adj matrix to actual marix dimensions
        # Create full-size adjacency matrix for correct_mst
        nodelen = len(nodelist)
        mst_adj_graph_correct = np.ndarray((nodelen, nodelen), np.bool)*False # T_STAR
        for i in range(mst_adj_graph_correct_0.shape[0]):
            for j in range(mst_adj_graph_correct_0.shape[1]):
                mst_adj_graph_correct[nodelist_to_correct_mapping[i], nodelist_to_correct_mapping[j]] = \
                mst_adj_graph_correct_0[i, j]

        # print(WScalarMat)

        """ For each node - Find MST with that source"""
        dLdOut = np.zeros(WScalarMat.shape)
        
        worst_Tx = None
        worst_graph_dist = 0
        
#         best_Tx = None
#         best_graph_dist = np.inf
        
        for source in range(len(nodelist)):
            (mst_nodes, mst_adj_graph) = MST(nodelist, WScalarMat, conflicts_Dict, source) # T_X
            # print('.', end = '')
            
            graph_dist = np.sum(mst_adj_graph&(~mst_adj_graph_correct))
            if graph_dist > worst_graph_dist:
                worst_graph_dist = graph_dist
                worst_Tx = mst_adj_graph
#             if graph_dist < best_graph_dist:
#                 best_graph_dist = graph_dist
#                 best_Tx = mst_adj_graph
            # Energy diff should all be negative
#             print('Source: [{}], Graph_Dist: {}, Energy: {:.2f}, Energy Diff: {:.2f}'.format(source, graph_dist,  np.sum(WScalarMat[mst_adj_graph]),\
#                                                                 np.sum(WScalarMat[mst_adj_graph_correct]) - np.sum(WScalarMat[mst_adj_graph])))
            
            # Total_Loss += (W_star - GetMSTWeight(mst_nodes, WScalarMat))
            Total_Loss += GetLoss(mst_adj_graph, mask_de_correct_edges, WScalarMat)

        """ Gradient Descent """
        # FOR MOST OFFENdING Y
        doBpp = False
        worst_energy_diff = np.sum(WScalarMat[mst_adj_graph_correct]) - np.sum(WScalarMat[worst_Tx])
        if M_hinge + worst_energy_diff > 0:
            doBpp = True
            if not self.neuralnet.outer_relu:
                # For new loss function with sigmoid on neuralnet
                dLdOut_inner = (1 - SigmoidGateOutput)
                dLdOut_inner[~mst_adj_graph] = 0 # Edges not in the current tree
                dLdOut_inner[mst_adj_graph^mask_de_correct_edges] = -dLdOut_inner[mst_adj_graph^mask_de_correct_edges]

                dLdOut += dLdOut_inner
            else:
                # For new loss function with ReLU on neuralnet
                dLdOut_inner = np.zeros_like(WScalarMat)
                
                dLdOut_inner[mst_adj_graph_correct] = 1
                dLdOut_inner[worst_Tx] -= 1

                dLdOut += dLdOut_inner
                
#         best_energy_diff = np.sum(WScalarMat[best_Tx]) - np.sum(WScalarMat[mst_adj_graph_correct])
#         if M_hinge + best_energy_diff > 0:
#             doBpp = True
#             if not self.neuralnet.outer_relu:
#                 # For new loss function with sigmoid on neuralnet
#                 dLdOut_inner = (1 - SigmoidGateOutput)
#                 dLdOut_inner[~mst_adj_graph] = 0 # Edges not in the current tree
#                 dLdOut_inner[mst_adj_graph^mask_de_correct_edges] = -dLdOut_inner[mst_adj_graph^mask_de_correct_edges]

#                 dLdOut += dLdOut_inner
#             else:
#                 # For new loss function with ReLU on neuralnet
#                 dLdOut_inner = np.zeros_like(WScalarMat)
                
#                 dLdOut_inner[best_Tx] = 1
#                 dLdOut_inner[mst_adj_graph_correct] -= 1
                

#                 dLdOut += dLdOut_inner

        if doBpp:
            print('{}. '.format(sentenceObj.sent_id), end = '')
            self.neuralnet.Back_Prop(dLdOut, len(nodelist), featVMat)
        else:
            trainingStatus[sentenceObj.sent_id] = True # Means tranining done on this file
            #print('Don\'t do BPP')
            pass
        
        Total_Loss /= len(nodelist)
        self.history[sentenceObj.sent_id].append(Total_Loss)
#         print("\nFileKey: %s, Loss: %6.3f, Loss2: %6.3f" % (sentenceObj.sent_id, Total_Loss, Loss_2))

In [39]:
# trainer.neuralnet.U

In [88]:
trainer = None
def InitModule(_matDB):
    global WD, trainer
    WD.word_definite_extInit(_matDB)
    trainer = Trainer()
InitModule(matDB)
trainingStatus = defaultdict(lambda: bool(False))
# trainer.neuralnet = pickle.load(open('outputs/train_nnet_t9543323920940.p', 'rb'))['nnet']

443


In [67]:
"""
################################################################################################
################################################################################################
################################################################################################
"""
# if __name__ == '__main__':
# loaded_SKT = pickle.load(open('../Simultaneous_CompatSKT_10K.p', 'rb'))
# loaded_DCS = pickle.load(open('../Simultaneous_DCS_10K.p', 'rb'))
# main(loaded_SKT, loaded_DCS)
np.set_printoptions(suppress=False)
trainer.Train(loaded_SKT['170095.p2'], loaded_DCS['170095.p2'])
trainer.Test(loaded_SKT['170095.p2'], loaded_DCS['170095.p2'])
# print ("Not Implemented")

170095. Max(delW):   0.000272	Max(delU):   0.000003


(8, 9, 9, 9)

In [ ]:
# Complete Training
trainingStatus = defaultdict(lambda: bool(False)) # Reset it after 3 epochs of full-training set
train(loaded_SKT, loaded_DCS, 4000)
playBeep()

Batch:  0
['141141.p2', '206744.p2', '30352.p2', '43009.p2', '5774.p2', '205691.p2', '216192.p2', '352667.p2', '59768.p2', '35561.p2', '433461.p2', '54567.p2', '165647.p2', '32144.p2', '410602.p2', '289063.p2', '236819.p2', '347229.p2', '24734.p2', '147078.p2']
ITERATION IN 0
141141. Max(delW):   0.000036	Max(delU):   0.000017
206744. Max(delW):   0.000085	Max(delU):   0.000068
30352. Max(delW):   0.000147	Max(delU):   0.000183
43009. Max(delW):   0.000061	Max(delU):   0.000054
5774. Max(delW):   0.000082	Max(delU):   0.000173
205691. Max(delW):   0.000118	Max(delU):   0.000126
216192. Max(delW):   0.000113	Max(delU):   0.000247
433461. Max(delW):   0.000129	Max(delU):   0.000172
54567. Max(delW):   0.000065	Max(delU):   0.000046
32144. Max(delW):   0.000029	Max(delU):   0.000053
289063. Max(delW):   0.000094	Max(delU):   0.000075
236819. Max(delW):   0.000109	Max(delU):   0.000165
347229. Max(delW):   0.000118	Max(delU):   0.000139
24734. Max(delW):   0.000102	Max(delU):   0.000251
IT

In [104]:
# Test on training set
test(loaded_SKT, loaded_DCS, n_testSet=500, _testFiles=TrainFiles)
playBeep()

0  Checkpoint... 
Failed!
100  Checkpoint... 
200  Checkpoint... 
300  Checkpoint... 
400  Checkpoint... 
Micro Recall[lemma]: 81.17824773413898
Micro Recall[word]: 65.95166163141994
Micro Precision[lemma]: 69.75597092419522
Micro Precision[word]: 56.67185877466251


In [ ]:
# POST TEST 1K SET
test(loaded_SKT, loaded_DCS, n_testSet = 1000)
playBeep()

In [101]:
print(p_name)
pickle.dump({
        'U': trainer.neuralnet.U,
        'W': trainer.neuralnet.W,
        'n': trainer.neuralnet.n,
        'd': trainer.neuralnet.d
    }, open(p_name, 'wb'))

outputs/train_nnet_t9745407094833.p


In [96]:
trainer.neuralnet.W

array([[ 0.00722495,  0.00338938,  0.00176284, ...,  0.009796  ,
         0.0068291 , -0.00310815],
       [ 0.00205929, -0.00027124, -0.00189266, ...,  0.00036876,
        -0.00670928,  0.01044982],
       [-0.00470419,  0.00519006, -0.00487317, ...,  0.00630901,
         0.00631652, -0.0028062 ],
       ..., 
       [-0.00981885, -0.00075694,  0.00930562, ..., -0.0052305 ,
        -0.00142855,  0.00257117],
       [-0.00283655,  0.00643924, -0.00442686, ...,  0.00092071,
        -0.00016554, -0.00016706],
       [-0.00847906, -0.00114583,  0.00118957, ..., -0.00622475,
         0.0050656 , -0.00734032]])

In [97]:
trainer.neuralnet.U

array([[ 0.04898352],
       [ 0.07198067],
       [ 0.04638693],
       [ 0.02759166],
       [ 0.08254517],
       [-0.08328421],
       [-0.03602132],
       [-0.04564724],
       [ 0.05644355],
       [-0.03174373],
       [-0.05054378],
       [-0.01256383],
       [ 0.06218375],
       [ 0.07763085],
       [-0.09395439],
       [-0.05032174],
       [-0.06160398],
       [-0.04385358],
       [ 0.03797409],
       [ 0.02710784],
       [ 0.06879289],
       [-0.02271876],
       [-0.0960755 ],
       [-0.01852471],
       [ 0.07529398],
       [ 0.04353588],
       [ 0.05879214],
       [ 0.09574614],
       [ 0.03866831],
       [-0.08346177],
       [-0.01153538],
       [-0.03091819],
       [-0.01030404],
       [ 0.05507836],
       [-0.02011198],
       [ 0.01430288],
       [-0.03512014],
       [-0.01738083],
       [ 0.09215843],
       [-0.08607582],
       [ 0.08938146],
       [-0.07966959],
       [ 0.06414074],
       [ 0.06171342],
       [ 0.0094823 ],
       [-0

3